In [ ]:
#final project " Geoprocessing and Buffers to Evaluate the Effect of Road Proximity on Vegetation"

In [1]:
!conda install -c conda-forge geopandas -y

Channels:
 - conda-forge
 - defaults
 - esri
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\bjetra\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone

  added / updated specs:
    - geopandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.7.2               |     pyhd8ed1ab_0          28 KB  conda-forge
    certifi-2024.8.30          |     pyhd8ed1ab_0         160 KB  conda-forge
    click-plugins-1.1.1        |             py_0           9 KB  conda-forge
    cligj-0.7.2                |     pyhd8ed1ab_1          10 KB  conda-forge
    fiona-1.9.5                |          py311_4        16.7 MB  esri
    folium-0.18.0              |     pyhd8ed1ab_0          77 KB  conda-forge
    geopandas-0.14.4           |     pyhd8ed1ab_0           8 KB  conda-forge
    geopandas-base-0.14.4      |     pyha770c72_0         99

In [ ]:


import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

#load roads and vegetation data
roads = gpd.read_file(r"C:\Users\bjetra\Downloads\data1\data1\Lesson1\us_roads.shp")
vegetation = gpd.read_file(r"Enter your vegetation shape file path")

#Reproject both datasets to the same CRS (e.g., EPSG:4326)

roads = roads.to_crs(epsg=4326)
vegetation = vegetation.to_crs(epsg=4326)


In [ ]:
#Buffer Zones Creation and Shapefile:
roads_buffer_100 = roads.buffer(100)
roads_buffer_500 = roads.buffer(500)
roads_buffer_1000 = roads.buffer(1000)

roads_buffer_100.to_file('roads_buffer_100m.shp')
roads_buffer_500.to_file('roads_buffer_500m.shp')
roads_buffer_1000.to_file('roads_buffer_1000m.shp')


In [ ]:
#Intersect Vegetation Data with Buffer Zones
intersection_100m = gpd.overlay(vegetation, roads_buffer_100, how='intersection')
intersection_500m = gpd.overlay(vegetation, roads_buffer_500, how='intersection')
intersection_1000m = gpd.overlay(vegetation, roads_buffer_1000, how='intersection')

In [ ]:
# Extract NDVI values from the intersection data
ndvi_values_100m = intersection_100m['NDVI']  
ndvi_values_500m = intersection_500m['NDVI'] 
ndvi_values_1000m = intersection_1000m['NDVI']

In [ ]:
# Statistical analysis can be used to determine whether vegetation health is considerably impacted by road proximity

f_value, p_value = stats.f_oneway(ndvi_values_100m, ndvi_values_500m, ndvi_values_1000m)

# Output the results of the ANOVA test
print(f"F-Value: {f_value}")
print(f"P-Value: {p_value}")

In [ ]:
# Calculate the mean NDVI for each buffer zone
mean_ndvi_100 = np.mean(ndvi_values_100m)
mean_ndvi_500 = np.mean(ndvi_values_500m)
mean_ndvi_1000 = np.mean(ndvi_values_1000m)

In [ ]:
#Visualization(To show how close roads are to vegetation, overlay buffer zones on a base map. we can Show how the vegetation health (such as the NDVI) varies within each buffer zone using color-coded maps and also make line plots or bar charts to show how vegetation health and road proximity relate to one another.)

buffer_distances = [100, 500, 1000]
ndvi_means = [mean_ndvi_100, mean_ndvi_500, mean_ndvi_1000]

plt.plot(buffer_distances, ndvi_means, marker='o')
plt.xlabel('Distance from Road (m)')
plt.ylabel('Mean NDVI')
plt.title('Vegetation Health vs. Distance from Road')
plt.show()